<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/ner_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip -q install transformers
!pip -q install sentencepiece

     |████████████████████████████████| 1.8MB 3.0MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 2.9MB 22.9MB/s 
     |████████████████████████████████| 1.2MB 2.0MB/s 


In [2]:
import pandas as pd
import numpy as np
from transformers import (
    pipeline, 
    AutoConfig, 
    AutoTokenizer, 
    AutoModel, 
    AutoModelForTokenClassification
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model = AutoModelForTokenClassification.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model.eval()
print()

In [4]:
albert_ner = pipeline('ner', model=model, tokenizer=tokenizer)

In [5]:
! rm -rf data
! wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data{1..10}.txt
! mkdir data && mv persian-ner* data/

persian-ner-twitter 100%[===================>] 276.22K  --.-KB/s    in 0.08s   
persian-ner-twitter 100%[===================>] 245.48K  --.-KB/s    in 0.02s   
persian-ner-twitter 100%[===================>] 308.37K  --.-KB/s    in 0.03s   
persian-ner-twitter 100%[===================>] 226.92K  --.-KB/s    in 0.02s   
persian-ner-twitter 100%[===================>] 296.95K  --.-KB/s    in 0.03s   
persian-ner-twitter 100%[===================>] 217.98K  --.-KB/s    in 0.02s   
persian-ner-twitter 100%[===================>] 287.11K  --.-KB/s    in 0.02s   
persian-ner-twitter 100%[===================>] 263.38K  --.-KB/s    in 0.02s   
persian-ner-twitter 100%[===================>] 298.06K  --.-KB/s    in 0.03s   
persian-ner-twitter 100%[===================>] 103.19K  --.-KB/s    in 0.006s  


# Convert to Text

In [6]:
from pathlib import Path
import re

def convert_lines_to_text(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [7]:
texts, tags = convert_lines_to_text('data/persian-ner-twitter-data1.txt')

In [12]:
print(texts[0])
print(tags[0])

['خرداد', '1399', '«', 'کیهان', '»', 'میراث', 'سناتور', 'مصطفی', 'مصباح\u200cزاده', '78', 'ساله', 'شد', 'کیهان', 'لندن', 'تنها', 'رسانه', 'ایرانی', 'در', 'تبعید', 'است', 'که', '«', 'از', 'جنگ', 'دوم', 'جهانی', 'تاکنون', 'قدمت', 'و', 'یک', 'موسسه', 'غصب', 'شده', 'در', 'تهران', '»', 'دارد']
['O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'B-LOC', 'O', 'O', 'B-NAT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVE', 'I-EVE', 'I-EVE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O']


In [36]:
s = ''
for word in texts[0]:
  s += word + ' '
s[84:90]

'ایرانی'

# Benchmark

In [31]:
def map_index2label(text, labels):
  index2label = {}
  start_index = 0
  for i, word in enumerate(text):
    end_index = start_index + len(word)
    index2label[(start_index, end_index)] =  labels[i]
    start_index = end_index + 1
  return index2label

In [37]:
index2label = map_index2label(texts[0], tags[0])

In [42]:
print()

خرداد 1399 « کیهان » میراث سناتور مصطفی مصباح‌زاده 78 ساله شد کیهان لندن تنها رسانه ایرانی در تبعید است که « از جنگ دوم جهانی تاکنون قدمت و یک موسسه غصب شده در تهران » دارد


In [ ]:
def benchmark(texts, labels_list, model, model_defined_labels):
  for text, labels in zip(texts, labels_list):
    index2label = map_index2label(text, labels)
    model_result = model(" ".join(text))
    start_end_index = [(item['start'], item['end']) for item in model_result]
    for index, label in index2label:
      if label in model_defined_labels:
        if index[0] 

    return

In [ ]:
benchmark(texts, tags, albert_ner, ["B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", ""])

In [ ]:
for idx, item in enumerate(tokenizer.tokenize(s)):
  print(item)
  if idx < len(texts[0]):
    print(texts[0][idx])
  print('----------')

In [ ]:
texts[0][]

'1399'

In [18]:
len("می‌خاره")

7

In [17]:
albert_ner("او می‌خاره مصباحزاده خره")
# for pred in pred_list:
#   print("word:", pred['word'])
#   print("entity:", pred['entity'])
#   print("index:", pred['index'])
  
#   print("s[start:end]:",s[pred['start']:pred['end']])
#   print("text[index]:",texts[0][pred['index']/2])
#   print("tags[index]:",tags[0][pred['index']])
#   print("-----------")

[{'end': 16,
  'entity': 'B-person',
  'index': 5,
  'score': 0.9936177730560303,
  'start': 11,
  'word': '▁مصباح'},
 {'end': 20,
  'entity': 'I-person',
  'index': 6,
  'score': 0.8189015984535217,
  'start': 16,
  'word': 'زاده'}]